In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/Abi_GenAI_Sessions"
)
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/Abi_GenAI_Sessions'

In [7]:
import json
import kaggle

In [8]:
# read a json file from .kaggle and set the environment variables
# os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
# os.environ['KAGGLE_KEY'] = 'your_kaggle_api_key'

with open(".kaggle/kaggle.json") as f:
    kaggle_json = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_json["username"]
os.environ["KAGGLE_KEY"] = kaggle_json["key"]

In [12]:
kaggle.api.dataset_download_files(
    "ishikajohari/taylor-swift-all-lyrics-30-albums",
    path="./data",
    unzip=True,
)

Dataset URL: https://www.kaggle.com/datasets/ishikajohari/taylor-swift-all-lyrics-30-albums


In [13]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset

2024-09-25 09:07:23.965330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 09:07:23.965376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 09:07:23.966251: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 09:07:23.972663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 09:07:24.870320: W tensorflow/compiler/tf2

Initial data processing

In [20]:
data_path = "data/data/Albums"
lyrics_files = glob.glob(f"{data_path}/**/*.txt", recursive=True)
print(len(lyrics_files))

879


In [21]:
def read_and_join_txt_files(file_paths):
    contents = []
    for path in file_paths:
        with open(path, 'r', encoding='utf-8') as file:
            contents.append(file.read())
    return '\n\n'.join(contents)

In [22]:
joined_content = read_and_join_txt_files(lyrics_files)

In [23]:
print(joined_content[:1000])

104 ContributorsTranslationsEspañolPortuguêsFrançaisClean Lyrics[Verse 1]
The drought was the very worst (Oh-oh, oh-oh)
When the flowers that we'd grown together died of thirst
It was months and months of back and forth (Oh-oh, oh-oh)
You're still all over me
Like a wine-stained dress I can't wear anymore

[Pre-Chorus]
Hung my head as I lost the war
And the sky turned black like a perfect storm
[Chorus]
Rain came pouring down
When I was drowning, that's when I could finally breathe
And by morning
Gone was any trace of you, I think I am finally clean
(Oh, oh, oh, oh)

[Verse 2]
There was nothing left to do (Oh-oh, oh-oh)
When the butterflies turned to
Dust that covered my whole room
So I punched a hole in the roof (Oh-oh, oh-oh)
Let the flood carry away all my pictures of you

[Pre-Chorus]
The water filled my lungs, I screamed so loud
But no one heard a thing

[Chorus]
Rain came pouring down
When I was drowning, that's when I could finally breathe
And by morning
Gone was any trace of yo

In [24]:
# Cleaning the file by removing/replacing unnecessary characters and removing sections 
# that are not lyrics
replace_with_space = ['\u2005', '\u200b', '\u205f', '\xa0', '-']
replace_letters = {'í':'i', 'é':'e', 'ï':'i', 'ó':'o', ';':',', '‘':'\'', '’':'\'', ':':',', 'е':'e'} 
remove_list = ['\)', '\(', '–','"','”', '"', '\[.*\]', '.*\|.*', '—']

cleaned_lyrics = joined_content

for old, new in replace_letters.items():
    cleaned_lyrics = cleaned_lyrics.replace(old, new)
for string in remove_list:
    cleaned_lyrics = re.sub(string,'',cleaned_lyrics)
for string in replace_with_space:
    cleaned_lyrics = re.sub(string,' ',cleaned_lyrics)
print(''.join(sorted(set(cleaned_lyrics))))

	
 !#$&'*,./0123456789?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz¦ÃÅÇÜáâãåçêñöüăčšΕάήαεηικλμνρστφМРСУагдийклнопрсуьїابةتجرسعفلمهيیदनहिी्ทปภยลษาแไᜄᜆᜎᜓ᜔ếệ‌‎“†…中文日本翻語译국어한


In [25]:
# Setting aside a portion for training the model and a portion for testing the data to prevent 
# the model from overfitting to the data it is tested on
split_point = int(len(cleaned_lyrics)*0.95)
train_data = cleaned_lyrics[:split_point]
test_data = cleaned_lyrics[split_point:]
train_data_seg = []
for i in range(0, len(train_data), 500):
        text = train_data[i:min(i+500, len(train_data))]
        train_data_seg.append(text)
train_data_seg = Dataset.from_dict({'text':train_data_seg})
print(len(train_data_seg))

3734


In [26]:
from huggingface_hub import notebook_login
notebook_login()

In [27]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

In [29]:
# Loading the model with double quantization and 4-bit quantization for the model weights and activations
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [30]:
# Loading the model with the quantization configuration and setting the device map to auto
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

/anaconda/envs/finetune_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:983: UserWarning: Not enough free disk space to download the file. The expected file size is: 4400.22 MB. The target location /home/azureuser/.cache/huggingface/hub/models--PY007--TinyLlama-1.1B-step-50K-105b/blobs only has 2906.28 MB free disk space.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device